In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n, get_bday
from gioutils.gui.base import View, ViewStore, WidgetView, Tabs
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer, RepeatedTimerThread
from gioutils import ezutils as ez
from gioutils.ezutils import bh, bhs, SQL, ZCILSHelper, get_price_info, get_id, LiveInflLocal, COBInflLocal
from gioutils.gui.base import ViewStore
from gioutils.blpw import BlpParser
from gioutils.gui.datagrid import DataGridGio

from api.data.base import data_path, radar_path
from api.gui.params import bbg_params, spot_params, fwd_params
from api.gui.theme import bg_color, grid_style
from api.collectors import CollectorStoreMixin, SpotForward, Futures, Market, Mtgs, FwdMatrix, VolMatrix, ECOPred, ZCILMatrix, ECO
from api.gui.editable_grid import EditableGrid
from api.gui.price_grids import PricingGrid
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.global_config import GlobalConfig
from api.gui.monitors import MktMonitor, VolMonitor, ECOMonitor, MktMonitorMenashe
from api.app import DataCollector
from api.gui.renderers import RendererStoreMixin, RendererComponentStoreMixin


pd.options.plotting.backend = "plotly"

In [3]:
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()

In [4]:
ccys = [
    'EUR', 
    'USD', 
    'GBP',
    # 'JPY'
]

In [5]:
bh.bhReadMergedTsCurves?

Signature:
bh.bhReadMergedTsCurves(
    dbhandle=None,
    tscrvhandle=None,
    startdate=None,
    enddate=None,
    market=None,
    curvename=None,
    source=None,
    category=None,
    _class=None,
    curvesdbname=None,
    curvesdbserver=None,
    histdbname=None,
    histdbserver=None,
    minextraptnr=None,
)
Docstring:
bhReadMergedTsCurves(dbhandle=None,tscrvhandle=None,startdate=None,enddate=None,market=None,curvename=None,source=None,category=None, _class=None,curvesdbname=None,curvesdbserver=None,histdbname=None,histdbserver=None,minextraptnr=None)

Read a time series of curves from Curves Db and Histories (for older curves), and merge them
  dbhandle - result db handle
  tscrvhandle - result tsCurve stub
  startdate - start date
  enddate - end date
  market - market name (e.g. COB)
  curvename - curve name to read from Curves Db (e.g. USD.3M.STD)
  source - histories source (e.g. CRV)
  category - histories category (e.g. O)
   _class - histories class (e.g BHCurve)
  

In [5]:
pricer_grid = PricingGrid()
mm = MktMonitorMenashe()
vm = VolMonitor()
em = ECOMonitor()

In [6]:
eco_fci = ECOPred(
    X_names= [
            'GT2 Govt',
            'GT10 Govt',
            'ES1 Index',
            'CDX HY CDSI GEN 5Y SPRD Curncy',
            'CDX IG CDSI GEN 5Y SPRD Curncy',
            'CL1 Comdty',
        ],
    y_name = 'NFCIINDX Index',
    name='nfci',
    display_name='NFCI',
    
)

In [7]:
sf = SpotForward(spot_params=spot_params, fwd_params=fwd_params, name='sf')
pg = pricer_grid = PricingGrid(name='pg')
fut = Futures(name='fut')
mkt = Market(name='mkt')
mtgs = Mtgs(name='mtgs')
mat = FwdMatrix(name='mat'),
vol_mat = VolMatrix(name='vol_mat')
zcil_mat = ZCILMatrix(name='zcil_mat')
eco = ECO(name='eco')

In [8]:
live_infl = LiveInflLocal()
cob_infl = COBInflLocal()

In [9]:
dc = DataCollector(
    live_infl=live_infl,
    cob_infl=cob_infl,
    ccys=None,
    bq=bq,
    #use_threads=True
)

In [10]:
gc = GlobalConfig(dc=dc, timer=RepeatedTimerThread)

In [11]:
tabs = Tabs(
    {
        'radar': v.Container(
            fluid=True,
            children=[
                v.Row(children=[v.Col(children=[gc.view])]),
                v.Row(children=[v.Col(children=[mm.view])])
            ],
        ),
        'radar':v.Container(
            fluid=True,
            children=[mm.view]
        ),
        'vol':v.Container(
            fluid=True,
            children=[vm.view]
        ),
        'eco':  v.Container(
            fluid=True,
            children=[em.view]
        ),
        'pricer': v.Container(
            fluid=True,
            children=[w.VBox([pricer_grid.box, pricer_grid.price_grid.view])],
        ),
        'monitor': pricer_grid.expr_dg.view,
        # 'a': w.HTML('test'),
        # 'b': w.HTML('test'),
        # 'c': w.HTML('test'),
    },
    force_grid_render=True,
    # tabs_kwargs=dict(vertical=True)
    # tabs_kwargs={'class_': "mb-2"}
)

In [12]:
js_fn = """
var el_ = document.querySelectorAll('[aria-label="notebook content"]');
console.log(el_)
el_[0].scrollTo(0, 0)
"""

In [13]:
js_fn = """
window.scrollTo(0, 100)
"""

In [14]:
import ipyvuetify as v 
import ipywidgets as w 
from IPython.display import Javascript

class Btn(v.Btn):
    """
    Custom textField that provide a handy copy to clipboard javascript behaviour. 
    When the clipboard btn is clicked the v_model will be copied in the local browser clipboard
    """
    output = w.Output()
    
    def __init__(self, js=None, **kwargs):
        
        self.js = js
        self.v_model = None
        super().__init__(**kwargs)
        
        # js behaviour
        if js:
            self.on_event('click', self.execute_js)
    
    @output.capture(clear_output=True)
    def execute_js(self, widget, event, data):
        """copy the content of the textfield in the clipboard"""
        js =  self.js
        display(Javascript(js))
        # display(self)
        return self
    
    def _ipython_display_(self):
        display(super(), self.output)

In [15]:
class Btn(v.VuetifyTemplate):

    def __init__(
        self,
        js,
        **kwargs
    ):
        self.btn = v.Btn(**kwargs)
        self._js = js
        self.template = f'''<myBtn/>
        <script> {{
            methods: {{
                jupyter_method(_txt) {{{js}}}
            }}
        }}
        </script>'''
        self.components={'myBtn': self.btn}
        super().__init__()
        
        def _fn(widget, event, data):
            self.send(
                {
                    "method": "method", 
                    # "args": [self.tf.v_model]
                }
            )
        self.btn.on_event('click', _fn)

In [16]:
btn = Btn(js=js_fn)

In [17]:
v.Container(
    children=[
        # gc.view,
        v.Row(
            children=[
                v.Col(children=[gc.view], offset=6, class_='my-0 py-0'),
            ],
        ),
        v.Row(
            children=[
                v.Col(children=[tabs.view], class_='ma-0 pa-0'),
            ],
        ),
        v.Row(
            children=[
                v.Col(children=[btn], class_='ma-0 pa-0'),
            ],
        ),
    ],
    fluid=True,
    class_='ma-0 pa-0',
    id='app-radar'
)

Container(children=[Row(children=[Col(children=[Row(children=[Col(children=[TextField(clearable=True, dense=Tr…